In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

In [ ]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"

In [ ]:
!pip install fasttext

In [ ]:
#!wget  https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/embedding-v2/indicnlp.ft.hi.300.bin

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.zip

In [ ]:
!unzip '/content/wiki.hi.zip'

In [ ]:
import fasttext
embedding_model = fasttext.load_model('/content/wiki.hi.bin')

In [ ]:
cd /content/calling-out-bluff/Model3(SkipFlow)/

In [ ]:
import  keras.layers  as  klayers 
from keras.preprocessing.text import text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
import numpy as np
import nltk
from quadratic_weighted_kappa import QWK
from sklearn.metrics import cohen_kappa_score
from keras.models import Model
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.optimizers import adam_v2
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import initializers
from scipy import stats

In [ ]:
EMBEDDING_DIM=300
MAX_NB_WORDS=4000

MAX_SEQUENCE_LENGTH=500
VALIDATION_SPLIT=0.20
DELTA=20

texts=[]

labels=[]


originals = []

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Untitled spreadsheet - Sheet1 (2).csv')

In [ ]:
df.head()

In [ ]:
df = df.drop('Unnamed: 18',axis=1)
df = df.dropna()

In [ ]:
for i in range(len(df)):
  originals.append(df['score'].iloc[i])

In [ ]:
essay_type = '1'

fp=open("/content/drive/MyDrive/promt_1_1783 - Sheet1.tsv",'r', encoding="utf-8", errors="ignore")
fp.readline()
originals = []

In [ ]:
for line in fp:
    temp=line.split("\t")
    if(temp[2]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        originals.append(float(temp[4]))
# print(originals)
fp.close()
# print(originals)
print("range min - ", min(originals) , " ; range max - ", max(originals))


In [ ]:
print("range min - ", min(originals) , " ; range max - ", max(originals))

In [ ]:
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

In [ ]:
from indicnlp.tokenize import indic_tokenize  

range_min = min(originals)
range_max = max(originals)
sentences=[]
for i in range(len(df)):
  texts.append(df['essay_hindi'].iloc[i])
  labels.append(((df['score'].iloc[i]-range_min)/(range_max-range_min)))
  line = df['essay_hindi'].iloc[i].strip()
  sentences.append(indic_tokenize.trivial_tokenize(line))

In [ ]:
from indicnlp.tokenize import indic_tokenize  

range_min = min(originals)
range_max = max(originals)

fp=open("/content/drive/MyDrive/promt_1_1783 - Sheet1.tsv",'r', encoding="utf-8", errors="ignore")
fp.readline()
sentences=[]
for line in fp:
    temp=line.split("\t")
    if(temp[2]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        texts.append(temp[5])
        labels.append((float(temp[4])-range_min)/(range_max-range_min)) ## why ??  - normalize to range [0-1]
        line=temp[5].strip()
        sentences.append(indic_tokenize.trivial_tokenize(line))

fp.close()


In [ ]:
tokenizer=Tokenizer(num_words = MAX_NB_WORDS) #num_words=MAX_NB_WORDS) #limits vocabulory size
tokenizer.fit_on_texts(texts) #encoding the text
sequences=tokenizer.texts_to_sequences(texts) #returns list of sequences
word_index=tokenizer.word_index #dictionary mapping, word and specific token for that word...
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #padding to max_length


print('Shape of data tensor:', data.shape)


In [ ]:
print("text labels appended %s" %len(texts))

labels=np.asarray(labels)
print(labels)
print(len(labels))

In [ ]:
fast_emb = {}
for i in sentences:
  temp1 = np.zeros((1, EMBEDDING_DIM))
  for w in i:
    #print(w,np.asarray([float(j) for j in embedding_model[w]]))
    fast_emb[w] = embedding_model[w]

In [ ]:
len(fast_emb)

In [ ]:
indices=np.arange(data.shape[0]) #with one argument, start=0, step =1
print(data.shape)
np.random.shuffle(indices)
data=data[indices]
print(data.shape)
labels=labels[indices]
# np.reshape(labels, ())
print(labels.shape)
validation_size=int(VALIDATION_SPLIT*data.shape[0])
print(validation_size)

In [ ]:
x_train=data[:-validation_size] #data-validation data
print(x_train.shape)
# print(x_train)
# print(labels)
y_train=labels[:-validation_size]
# print(y_train.transpose)
print(y_train.shape)
# y_train = np.reshape(y_train, (1427, 1))
# print(y_train_new)
# print(y_train)
x_val=data[-validation_size:]
print(x_val.shape)
y_val=labels[-validation_size:]

In [ ]:
embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))
print(embedding_matrix.shape)

In [ ]:
for word,i in word_index.items():
	if(i>=len(word_index)):
		continue
	if word in fast_emb:
			embedding_matrix[i]=fast_emb[word]
vocab_size=len(word_index)
print(vocab_size)

In [ ]:
embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=True,
							trainable=False)
# print(embedding_layer.shape)
side_embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=False,
							trainable=False)

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam
model = Sequential()
forward_layer = LSTM(30, return_sequences=True)
backward_layer = LSTM(30, activation='relu', return_sequences=True,
                      go_backwards=True)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                        input_shape=inputshape)
model.add(Bidirectional(LSTM(20)))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(512,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1, activation='leaky_relu'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
model.compile(optimizer='rmsprop',
              loss="mean_squared_error",
              metrics=['MSE'])

In [ ]:
model.summary()

In [ ]:
# print(sf_1)
epochs = 100
# epochs = 1000
print(type(x_train))
# y_train = np.asarray(y_train)
print(type(y_train))

model.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_data=([x_val], y_val))

In [ ]:
y_pred=model.predict([x_val])
y_pred

In [ ]:
y_val_fin = [int(round(a*(range_max-range_min)+range_min)) for a in y_val]
print(y_val_fin)

In [ ]:
y_pred_fin =[int(round(a*(range_max-range_min)+range_min)) for a in y_pred.reshape(356).tolist()]
print(y_pred_fin)

In [ ]:
print(cohen_kappa_score(y_val_fin,y_pred_fin,weights="quadratic"))